# <center> Customer Purchase

# Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import warnings

%matplotlib inline
warnings.filterwarnings('ignore')

# Importing Data

In [ ]:
data = pd.read_csv('../input/cvdcvd-vd/Social_Network_Ads.csv')

In [ ]:
data = data.set_index('User ID')

In [ ]:
data.info()

In [ ]:
data.groupby(['Purchased','Gender']).median()

# EDA

In [ ]:
sns.countplot(data=data,x='Purchased')

In [ ]:
sns.countplot(data=data,x='Purchased',hue='Gender')

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
sns.histplot(data=data,hue='Purchased',x='Age',bins=20,element='poly')
plt.subplot(1,2,2)
sns.histplot(data=data,hue='Purchased',x='EstimatedSalary',bins=20,element='poly')

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
sns.boxplot(data=data,x='Purchased',y='Age',)
plt.subplot(1,2,2)
sns.boxplot(data=data,x='Purchased',y='EstimatedSalary')

In [ ]:
plt.figure(figsize=(15,7))
sns.scatterplot(data=data,x='Age',y='EstimatedSalary',hue='Purchased',style='Gender')

# One Hot Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
to_be_encoded = ['Gender']
label_encoder = LabelEncoder()
dfs = []
for i in to_be_encoded:
    temp = pd.DataFrame({'Before Encoding':data[i].unique(),'After Encoding':label_encoder.fit_transform(data[i].unique())})
    dfs.append([temp.sort_values(by=['After Encoding'],),i])
    data[i] = label_encoder.fit_transform(data[i])
print(dfs[0][1])
dfs[0][0]

# Feature Importance

In [ ]:
X = data.drop('Purchased',axis=1)

Y = data['Purchased']

In [ ]:
X['nf']=X['EstimatedSalary']*X['Age']
X['nf2']=X['EstimatedSalary']*X['EstimatedSalary']

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
fs = SelectKBest(score_func=chi2, k='all')
fs.fit(X, Y)
per = []
for i in fs.scores_:
    per.append(round(((i/sum(fs.scores_))*100),3))

features_data = pd.DataFrame({'Feature':X.columns,'Scores':fs.scores_,'Importance (%)':per}).sort_values(by=['Scores'],ascending=False)

plt.figure(figsize=(9,3))
sns.barplot( 'Importance (%)','Feature',orient='h',data=features_data)
insignificant = features_data.loc[features_data['Importance (%)']<0.005]['Feature'].unique()
features_data

In [ ]:
X = X.drop(insignificant,axis=1)

# Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

# Test Train Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=100)

# Model Selection and Estimation

In [ ]:
from sklearn.metrics import accuracy_score,classification_report

#XGB
import xgboost as xgb
from xgboost import XGBClassifier
xgb = XGBClassifier() 


# Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

#RFC
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

#KNN
from sklearn.neighbors import KNeighborsClassifier
accuracy = []
for i in range(1,40):    
    kn = KNeighborsClassifier(n_neighbors=i)
    kn.fit(X_train,Y_train)
    predK = kn.predict(X_test)
    accuracy.append([accuracy_score(Y_test,predK),i])
    #print('Tested for k =',i)
temp = accuracy[0]
for m in accuracy:
    if temp[0] < m[0]:
        temp=m
knn = KNeighborsClassifier(n_neighbors=temp[1])

#SVM
from sklearn.svm import SVC
svc = SVC()

from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.1,1, 10, 100, 1000,2000], 'gamma': [1,0.1,0.01,0.001,0.0001], 'kernel': ['rbf']} 
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=3)

print('Models Imported')

In [ ]:
model_acc = []
models = [xgb,lr,rfc,knn,svc,grid]
#model_name = ['xgb','lr','rfc','kno','svc','grid']
for i in models:
    i.fit(X_train,Y_train)
    model_acc.append(accuracy_score(Y_test,i.predict(X_test)))
                      
models = pd.DataFrame({'Models':models,'Accuracy':model_acc})

In [ ]:
models = models.sort_values(by=['Accuracy'],ascending=False).reset_index().drop('index',axis=1)
best = models['Models'][0]
models['Models']=models['Models'].astype(str).str.split("(", n = 2, expand = True)[0]
models

In [ ]:
print('Hence the best model is',models['Models'][0],'with an accuracy of',round((models['Accuracy'][0]*100),2),'%')
print('\nThe classification report is:')
print(classification_report(Y_test,best.predict(X_test)))